In [1]:
import os
import time
import datetime
from PIL import Image
import pickle
import pandas as pd
import tensorflow as tf
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms

In [2]:
cuda_gpu = torch.cuda.is_available()
if cuda_gpu:
    device = torch.device("cuda:0")
    torch.backends.cudnn.benchmark = True
else:
    device = torch.device("cpu:0")
nclass = 100

In [4]:
model = torchvision.models.vgg19_bn(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to C:\Users\70659/.cache\torch\hub\checkpoints\vgg19_bn-c79401a0.pth
100%|██████████| 548M/548M [00:49<00:00, 11.7MB/s]


In [3]:
model = torchvision.models.resnext101_32x8d(pretrained=True)
model.fc = nn.Linear(2048, 100, bias=True)
model = model.to(device)

In [4]:
for name, value in model.named_parameters():
    if "fc" not in name and "classifier" not in name:
        value.requires_grad = False
# for name, param in model.named_parameters():
#     print(name,param.requires_grad)

In [5]:
EPOCHS = 30
BATCH_SIZE = 64
LR = 0.001
SAVE_EPOCH = 3
VAL_EPOCH = 6
verbose = 500
loss_func = nn.CrossEntropyLoss()

In [6]:
data_transformer = transforms.Compose([
    #transforms.RandomResizedCrop(64),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(256),
    transforms.ToTensor()
    #transforms.Normalize([124,117,104],[58.4,57.1,57.4])
    ])

class ImageNetData(Dataset):
    
    def __init__(self, path, info, nclass):
        self.path = path
        info_path = os.path.join(path, info)
        f = open(info_path)
        self.data = f.read().splitlines()
        self.transforms = data_transformer
        self.nclass = nclass
    
    def __len__(self):
        return len(self.data)
        
        
    def __getitem__(self, index):
        img = Image.open(os.path.join(self.path, self.data[index].split(' ')[0]))
        img = self.transforms(img)
        label = self.data[index].split(' ')[1]
        label = int(label)
#         one_hot = np.zeros(self.nclass, dtype=np.int)
#         one_hot[label] = 1
        return (img, label)

path = "D:/课程文件/大四上/深度学习导论/作业/5/TinyImageNet/TinyImageNet"
train_info_path = path + '/train.txt'
val_info_path = path + '/val.txt'
train_set = ImageNetData(path, "train.txt", nclass)
val_set = ImageNetData(path, "val.txt", nclass)

train_loader = DataLoader(dataset=train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(dataset=val_set, batch_size=BATCH_SIZE)      

In [7]:
ckptpath = "./ckpts/"
ckpts = sorted([x for x in os.listdir(ckptpath) if x.endswith(".ckpt")])
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
lr_reduct = lambda epochs: 0.5 ** (epochs // 20)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_reduct)
if len(ckpts) >= 1:
    state_dicts = torch.load(ckptpath+ckpts[-1])
    model.load_state_dict(state_dicts['model'])
    optimizer.load_state_dict(state_dicts['optimizer'])
    scheduler.load_state_dict(state_dicts['scheduler'])

In [8]:
torch.save(model, "resnext_fc_trained.pth")

In [8]:
train_loss = []
train_acc = []
val_loss = []
val_acc = []
with torch.autograd.set_detect_anomaly(True):
    for epochs in range(1, EPOCHS + 1):
        print('Epoch {}/{}'.format(epochs, EPOCHS))
        start_time = time.time()
        running_train_loss = []
        running_train_acc = []
        for i, data in enumerate(train_loader):
            inputs = data[0].to(device)
            targets = data[1].to(device)
            outputs = model(inputs)
            loss = loss_func(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pred = outputs.argmax(axis=1)
            running_train_loss.append(float(loss))
            running_train_acc.append(float((pred==targets).sum()) / BATCH_SIZE)
            if verbose > 0 and (i + 1) % verbose == 0:
                print("Train: {}/{} - Average Time: {}".format(i + 1, len(train_loader), (time.time()-start_time) / (i + 1)))
        print("End of Train Epoch: {} - Average Time: {}".format(epochs, (time.time() - start_time) / (i + 1)))
        # 周期性清除CUDA缓存
        torch.cuda.empty_cache()
        if epochs % SAVE_EPOCH == 0:
            torch.save({'model': model.state_dict(), 
                       'optimizer': optimizer.state_dict(), 
                       'scheduler': scheduler.state_dict()},
                       ckptpath + "model_{}.ckpt".format(int(datetime.datetime.now().timestamp())))
        if epochs % VAL_EPOCH == 0:
            running_val_loss = []
            running_val_acc = []
            for i, data in enumerate(val_loader):
                inputs=data[0].to(device)
                targets=data[1].to(device)
                with torch.no_grad():
                    outputs = model(inputs)
                    loss = loss_func(outputs, targets)
                    pred = outputs.argmax(axis=1)
                    running_val_loss.append(float(loss))
                    running_val_acc.append(float((pred==targets).sum()) / BATCH_SIZE)
                if verbose > 0 and(i + 1) % verbose == 0:
                    print("Val: {}/{}".format(i + 1, len(val_loader)))
        
        scheduler.step()    
        train_loss.append(sum(running_train_loss) / len(running_train_loss))
        train_acc.append(sum(running_train_acc) / len(running_train_acc))
        duration = time.time() - start_time
        print('Train loss : {} Train accuracy : {}'.format(train_loss[-1], train_acc[-1]))
        if epochs % VAL_EPOCH == 0:
            val_loss.append(sum(running_val_loss) / len(running_val_loss))
            val_acc.append(sum(running_val_acc) / len(running_val_acc))
            print('Val   loss : {} Val   accuracy : {}'.format(val_loss[-1], val_acc[-1]))
        print('{:.0f}m {:.0f}s'.format(duration // 60, duration % 60))

Epoch 1/30
Train: 500/1852 - Average Time: 1.4833536701202392
Train: 1000/1852 - Average Time: 1.4715055954456329
Train: 1500/1852 - Average Time: 1.4613547286987305
End of Train Epoch: 1 - Average Time: 1.4534766398547279
Train loss : 1.6001149354096236 Train accuracy : 0.7970062395008414
44m 52s
Epoch 2/30
Train: 500/1852 - Average Time: 1.4283644976615906
Train: 1000/1852 - Average Time: 1.4351588249206544
Train: 1500/1852 - Average Time: 1.4417788144747417
End of Train Epoch: 2 - Average Time: 1.447733206058939
Train loss : 1.5348703046741676 Train accuracy : 0.8036657067434625
44m 41s
Epoch 3/30
Train: 500/1852 - Average Time: 1.4582891778945923
Train: 1000/1852 - Average Time: 1.4674520030021667
Train: 1500/1852 - Average Time: 1.4695797085762023
End of Train Epoch: 3 - Average Time: 1.469803229650205
Train loss : 1.5037949961865598 Train accuracy : 0.8079253659707255
45m 23s
Epoch 4/30
Train: 500/1852 - Average Time: 1.4746369490623474
Train: 1000/1852 - Average Time: 1.47435124

KeyboardInterrupt: 

In [10]:
torch.save({'model': model.state_dict(), 
            'optimizer': optimizer.state_dict(), 
            'scheduler': scheduler.state_dict()},
            ckptpath + "model_{}.ckpt".format(int(datetime.datetime.now().timestamp())))

In [10]:
ckpts[-1]

'model_1604847063.ckpt'

In [12]:
time.localtime(1604877733)

time.struct_time(tm_year=2020, tm_mon=11, tm_mday=9, tm_hour=7, tm_min=22, tm_sec=13, tm_wday=0, tm_yday=314, tm_isdst=0)

In [8]:
class ImageNetDataTest(Dataset):
    def __init__(self, path, info, nclass):
        self.path = path
        info_path = os.path.join(path, info)
        f = open(info_path)
        self.data = f.read().splitlines()
        self.transforms = data_transformer
        self.nclass = nclass
    
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, index):
        img = Image.open(os.path.join(self.path, self.data[index].split(' ')[0]))
        img = self.transforms(img)
#         one_hot = np.zeros(self.nclass, dtype=np.int)
#         one_hot[label] = 1
        return (self.data[index], img)

test_info_path = path + '/test.txt'
test_set = ImageNetDataTest(path, "test.txt", nclass)
test_loader = DataLoader(dataset=test_set, batch_size=BATCH_SIZE, shuffle=False)

In [9]:
names = []
preds = []
preds_val = {}
for i, data in enumerate(test_loader):
    with torch.no_grad():
        name = data[0]
        inputs = data[1].to(device)
        outputs = model(inputs)
        preds_val[name] = outputs.cpu().numpy()
        pred = outputs.argmax(axis=1).squeeze()
        names.extend(name)
        preds.extend(pred.cpu().numpy())
    if (i + 1) % 100 == 0:
        print("Test: {}/{}".format(i + 1, len(test_set)))

KeyboardInterrupt: 

In [13]:
df = pd.DataFrame([names,preds]).transpose()
df.columns = ['Id', 'Category']
df.loc[:, 'Id']=df['Id'].apply(lambda x: x.split('/')[-1])

In [15]:
df.to_csv("submission_resnext.csv", index=False)

In [13]:
f = open("prediction_resnext.pkl", "wb")
pickle.dump(preds_val, f)
f.close()

In [8]:
def getPrediction(loader, fname, verbose, total):
    preds_val = []
    with torch.no_grad():
        for i, data in enumerate(loader):
            inputs = data[0].to(device)
            targets = data[1].to(device)
            outputs = model(inputs)
            preds_val.append((targets.cpu().numpy(), outputs.cpu().numpy()))
            if (i + 1) % verbose == 0:
                print("Predict: {}/{}".format(i + 1, total))
            torch.cuda.empty_cache()
    f = open(fname, "wb")
    pickle.dump(preds_val, f)
    f.close()

train_loader = DataLoader(dataset=train_set, batch_size=64, shuffle=False)
val_loader = DataLoader(dataset=val_set, batch_size=64, shuffle=False)

In [9]:
getPrediction(train_loader, "prediction_resnext_train.pkl", 100, int(len(train_set)/64))

Predict: 100/1562
Predict: 200/1562
Predict: 300/1562
Predict: 400/1562
Predict: 500/1562
Predict: 600/1562
Predict: 700/1562
Predict: 800/1562
Predict: 900/1562
Predict: 1000/1562
Predict: 1100/1562
Predict: 1200/1562
Predict: 1300/1562
Predict: 1400/1562
Predict: 1500/1562


In [10]:
getPrediction(val_loader, "prediction_resnext_val.pkl", 100, int(len(val_set)/64))

Predict: 100/156
